<a href="https://colab.research.google.com/github/olaviinha/inhagcutils/blob/master/inhagcutils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# inhagcutils

A set of modules and functions I seem to need in too many Colab Notebooks.

### Import modules

In [ ]:
import sys, os, ntpath, string, random, librosa, shutil
from google.colab import drive
from glob import glob
from os.path import isdir, join
from IPython.display import Image, Audio, display
import numpy as np
import matplotlib.pyplot as plt

### Common variables

In [ ]:
# Dirs
drive_root = "/content/drive/My Drive/"
dir_tmp = "/content/tmp"

# Verbose
ffmpeg_q = "-hide_banner -loglevel panic" # "", -hide_banner -loglevel panic
sox_q = "-q" # -S, -q
youtube_q = "-q" # "", -q

# Audio formats
wav_44 = "-c:a pcm_s16le -ar 44100 -ac 2 "
wav_48 = "-c:a pcm_s24le -ar 48000 -ac 2"
mp3_192 = "-vn -ar 44100 -ac 2 -b:a 192k"
mp3_320 = "-vn -ar 48000 -ac 2 -b:a 320k"

### Functions: general

In [ ]:
# Input: -
# Output (boolean): ipynb is running on _hosted_ Colab runtime
def isHostedRuntime():
  return 'google.colab' in sys.modules

# Input (str): path
# Output (string): path with missing / at the end
def fix_path(path):
  if os.path.isdir(path) and not path.endswith('/'):
    path = path+"/"
  return path

# Input (str): path
# Output (str): filename with extension
def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail or ntpath.basename(head)

# Input (str): path
# Output (str): filename without extension
def basename(path):
  filename = os.path.basename(path).strip()#.replace(" ", "_")
  filebase = os.path.splitext(filename)[0]
  return filebase

# Input (str): path
# Output (str): "dir", "file", "youtube", "link" or "unknown"
def check_input_type(path):
  if os.path.isdir(path):
    input_type = "dir"
    input = fix_path(path)
  elif os.path.isfile(path):
    input_type = "file"
  elif "http" in path and "youtu" in path:
    input_type = "youtube"
  elif "http" in path:
    input_type = "link"
  else:
    input_type = "unknown"
  return input_type

# Input (list): list of anything
# Output (any): single item value with the most frequent occurrance
def most_frequent(list):
  freq = max(set(list), key = list.count)
  print(str(list.count(freq))+' out of '+str(len(list)), 'items have a value of', str(freq))
  return freq

# Input (str): stupid filename, e.g. any long% weird !filename (like this).wav
# Output (str): cool filename, e.g. any_long_weird_filename_like_this.wav
def slug(s):
  valid_chars = "-_. %s%s" % (string.ascii_letters, string.digits)
  file = ''.join(c for c in s if c in valid_chars)
  file = file.replace(' ','_')
  return file

# Input (int): number
# Output (str): random string of <number> characters long
def rnd_str(length):
  letters = string.ascii_lowercase
  result_str = ''.join(random.choice(letters) for i in range(length))
  return result_str

### Functions: temporary holds

In [ ]:
# Input (list): list of directory paths
# Output: -
def create_dirs(paths):
  for path in paths:
    if not os.path.isdir(path):
      !mkdir "{path}"
      
# Input (list): list of directory paths
# Output: -
def remove_dirs(paths):
  for path in paths:
    if os.path.isdir(path):
      !rmdir -r "{path}"

# Input (list): list of directory paths
# Output: -
def reset_dirs(paths):
  remove_dirs(paths)
  create_dirs(paths)

### Functions: audio-related

In [ ]:
# Input (str): path
# Output (list): list of audiofiles in dir
def list_audio(path):
  audiofiles = []
  for ext in ('*.wav', '*.aiff', '*.aif', '*.caf' '*.flac', '*.mp3', '*.ogg'):
    audiofiles.extend(glob(join(path, ext)))
  return audiofiles

# Input (str): path to WAV file
# Output: audio waveform image
def waveform(audiofile):
  y, sr = librosa.load(audiofile, sr=None)
  plt.figure(figsize=(10, 4))
  plt.rcParams['figure.facecolor'] = 'black'
  librosa.display.waveplot(y, sr=sr, color='#dfd')
  plt.show()

# Input (str): path to MP3 file
# Output: audio player
def audio_player(audiofile):
  display(Audio(audiofile))

# Input (str): path to MP3 file, path to WAV file (of the same track)
# Output: audio waveform image and audio player
def waveform_player(mp3, wav):
  waveform(wav)
  audio_player(mp3)